<a href="https://colab.research.google.com/github/kimhwijin/HandsOnMachineLearing/blob/main/HOML_Exercise_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier

In [4]:
mnist = fetch_openml('mnist_784', version=1)
X = mnist.data
y = mnist.target.astype(np.uint8)
print(X.shape)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=10000, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=10000, random_state=42)

(70000, 784)


In [9]:
#8

random_forest_clf = RandomForestClassifier(n_estimators=100, random_state=42)
extra_trees_clf = ExtraTreesClassifier(n_estimators=100, random_state=42)
svm_clf = LinearSVC(max_iter=100, tol=20, random_state=42)
mlp_clf = MLPClassifier(random_state=42)

estimators = [random_forest_clf, extra_trees_clf, svm_clf, mlp_clf]
for estimator in estimators:
    print("Training the", estimator)
    estimator.fit(X_train, y_train)

[estimator.score(X_val, y_val) for estimator in estimators]

Training the RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)
Training the ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=None, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=100,
                 

[0.9692, 0.9715, 0.8662, 0.9639]

In [10]:
[accuracy_score(y_val, estimator.predict(X_val)) for estimator in estimators]

[0.9692, 0.9715, 0.8662, 0.9639]

In [21]:
named_estimators = [
    ("random_forest_clf", random_forest_clf),
    ("extra_trees_clf", extra_trees_clf),
    ("svm_clf", svm_clf),
    ("mlp_clf", mlp_clf),
]
voting_clf = VotingClassifier(named_estimators)
voting_clf.fit(X_train, y_train)
print(voting_clf.score(X_val, y_val))
print([estimator.score(X_val, y_val) for estimator in voting_clf.estimators_])

0.9702
[0.9692, 0.9715, 0.8662, 0.9639]


In [22]:
voting_clf.estimators_
voting_clf.set_params(svm_clf=None)
del voting_clf.estimators_[2]

voting_clf.voting = 'hard'
print(voting_clf.voting)
print(voting_clf.score(X_val, y_val))

voting_clf.voting = 'soft'
print(voting_clf.voting)
print(voting_clf.score(X_val, y_val))

hard
0.9736
soft
0.97


In [24]:
voting_clf.voting = "hard"
print("TEST ENSEMBLE : ", end='')
print(voting_clf.score(X_test, y_test))

print("TEST EACH : ", end='')
print([estimator.score(X_test, y_test) for estimator in voting_clf.estimators_])

TEST ENSEMBLE : 0.9704
TEST EACH : [0.9645, 0.9691, 0.9604]
